Address component extraction from the text. 

PS: Please select GPU as a runtime for fater processing. 

Owner: Chetan Khadke <khadkechetan@gmail.com>


In [ ]:
!pip install simpletransformers

In [2]:
import pandas as pd
import re, math
import numpy as np
import pandas as pd
from scipy.special import softmax

from simpletransformers.ner import NERModel
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [3]:
columns = ['Building_Name', 'Building_Number', 'City', 'Recipient',
       'Street_Name', 'Zip_Code', 'State', 'Country']

In [4]:
count=0
labels = set()
def generate_data(file_name):
    data = pd.read_csv(file_name)
    def get_value(x):
        global count
        results = []
        address = x["Address"]
        previous = ""
        _temp = x.to_dict()
        for each in address.split():
            match = False
            for col in columns:
                if each.strip(" ,.-") in str(_temp[col]):
                    if previous == col:
                        results.append((count, each, "I-"+col))
                        labels.add("I-"+col)
                    else:
                        results.append((count, each, "B-"+col)) 
                        labels.add("B-"+col)
                    previous = col
                    match = True
                    break
            if not match:
                results.append((count, each, "O")) 
                labels.add("O")
                match = False
        # results.append(("", "",""))
        count += 1
        return results
    values = data[:].apply(lambda x:get_value(x), axis=1)
    flat_list = [item for sublist in values for item in sublist]
    train_df = pd.DataFrame(flat_list, columns=["sentence_id", "words", "labels"])
    return train_df

In [5]:
train_df = generate_data("us-train-dataset.csv")

In [6]:
test_df = generate_data("us-test-dataset.csv")

In [7]:
train_df[:20]

,sentence_id,words,labels
0,0,19,B-Building_Number
1,0,ST,B-Street_Name
2,0,ANDREW,I-Street_Name
3,0,"ST,",I-Street_Name
4,0,"BULRINGTON,",B-City
5,0,"VT,",B-State
6,0,"05401,,",O
7,0,United,B-Country
8,0,States,I-Country
9,1,2574,B-Building_Number


In [8]:

train_df.to_csv(r'pandas1.txt', header=None, index=None, sep=' ', mode='a')

In [9]:
arguments={"save_eval_checkpoints": False,
      "save_steps": -1,
      'overwrite_output_dir': True,
      "save_model_every_epoch": False,
      'reprocess_input_data': True, 
      "train_batch_size": 5,
      'num_train_epochs': 50,
      "max_seq_length": 256, 
      "gradient_accumulation_steps": 8,
      "learning_rate": 4e-5,
      "do_lower_case": False,
      "adam_epsilon" : 1e-8,
      "train_batch_size": 8,
      "eval_batch_size": 8,
      "fp16" : True
      }

In [10]:
model = NERModel(
    "bert",
    "distilbert-base-uncased",
    args=arguments,
    labels =  list(labels)
)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertForTokenClassification: ['vocab_transform.bias', 'distilbert.transformer.layer.2.attention.k_lin.bias', 'distilbert.transformer.layer.5.attention.q_lin.weight', 'distilbert.transformer.layer.4.ffn.lin1.bias', 'distilbert.transformer.layer.2.ffn.lin2.weight', 'distilbert.transformer.layer.3.ffn.lin1.weight', 'distilbert.transformer.layer.1.attention.q_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.output_layer_norm.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.5.attention.k_lin.weight', 'distilbert.transformer.layer.1.ffn.lin2.weight', 'distilbert.transformer.layer.5.output_layer_norm.weight', 'distilbert.transformer.layer.3.output_layer_norm.weight', 'distilbert.transformer.layer.3.sa_layer_norm.bias', 'distilbert.transformer.layer.2.attention.v

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [11]:
labels

{'B-Building_Name',
 'B-Building_Number',
 'B-City',
 'B-Country',
 'B-Recipient',
 'B-State',
 'B-Street_Name',
 'B-Zip_Code',
 'I-Building_Name',
 'I-City',
 'I-Country',
 'I-Recipient',
 'I-Street_Name',
 'O'}

In [12]:
# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, predictions = model.eval_model(train_df)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Running Epoch 0 of 50:   0%|          | 0/45 [00:00<?, ?it/s]

Running Epoch 1 of 50:   0%|          | 0/45 [00:00<?, ?it/s]

Running Epoch 2 of 50:   0%|          | 0/45 [00:00<?, ?it/s]

...

Running Epoch 47 of 50:   0%|          | 0/45 [00:00<?, ?it/s]

Running Epoch 48 of 50:   0%|          | 0/45 [00:00<?, ?it/s]

Running Epoch 49 of 50:   0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/45 [00:00<?, ?it/s]

In [13]:
result

{'eval_loss': 0.11480890100614892,
 'precision': 0.9591054313099041,
 'recall': 0.9578813018506701,
 'f1_score': 0.9584929757343551}

In [14]:
result, model_outputs, predictions = model.eval_model(test_df)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
result

{'eval_loss': 0.7861490646998087,
 'precision': 0.7011494252873564,
 'recall': 0.7176470588235294,
 'f1_score': 0.7093023255813954}

In [16]:
test_df.head(10)

,sentence_id,words,labels
0,354,223,B-Building_Number
1,354,NW,B-Street_Name
2,354,STATE,I-Street_Name
3,354,"STRE,",I-Street_Name
4,354,"DOVER,",B-City
5,354,DE,B-State
6,354,"19001,",B-Zip_Code
7,354,United,B-Country
8,354,States,I-Country
9,355,216,B-Building_Number


In [17]:
predictions1 = [item for sublist in predictions for item in sublist]

In [18]:
test_df["predicted"] = predictions1

In [19]:
test_df.head(20)

,sentence_id,words,labels,predicted
0,354,223,B-Building_Number,B-Building_Number
1,354,NW,B-Street_Name,B-Street_Name
2,354,STATE,I-Street_Name,I-Street_Name
3,354,"STRE,",I-Street_Name,I-Street_Name
4,354,"DOVER,",B-City,B-City
5,354,DE,B-State,B-State
6,354,"19001,",B-Zip_Code,B-Zip_Code
7,354,United,B-Country,B-Country
8,354,States,I-Country,B-Country
9,355,216,B-Building_Number,B-Building_Number


In [24]:
# Predictions on arbitary text strings
sentences = ["I stay at HIGHWAY 11-E, RUSSELLVILLE, TN 00000, United States"]
predictions, raw_outputs = model.predict(sentences)


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
predictions

[[{'I': 'O'},
  {'stay': 'I-Street_Name'},
  {'at': 'O'},
  {'HIGHWAY': 'I-Street_Name'},
  {'11-E,': 'I-Street_Name'},
  {'RUSSELLVILLE,': 'B-City'},
  {'TN': 'B-State'},
  {'00000,': 'O'},
  {'United': 'B-Country'},
  {'States': 'I-Country'}]]